In [18]:
import os
import pdfplumber
import fitz
import re
import nltk
from nltk.tokenize import sent_tokenize

In [17]:
nltk.download('punkt')

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1028)>


False

In [25]:
def extract_text_from_pdf(file_path):
    text = ''
    with fitz.open(file_path) as pdf:
        for page in pdf:  # pdf.pages가 아니라 pdf 자체를 반복
            page_text = page.get_text()
            if page_text:
                text += page_text + '\n'
    return text

def extract_texts_from_folder(folder_path):
    all_text = ''
    for filename in os.listdir(folder_path):
        if filename.endswith('.pdf'):
            file_path = os.path.join(folder_path, filename)
            print(f"Extracting from {filename}...")
            text = extract_text_from_pdf(file_path)
            all_text += f"=== Start of {filename} ===\n"
            all_text += text + '\n\n'
    return all_text

folder_path = '/Users/eunsol/Documents/금융데이터'  # PDF들이 들어있는 폴더 경로
full_text = extract_texts_from_folder(folder_path)

# 결과를 텍스트 파일로 저장
with open('combined_text.txt', 'w', encoding='utf-8') as f:
    f.write(full_text)

print("모든 PDF 텍스트 추출 완료 및 combined_text.txt 저장됨")

Extracting from 은행산업 관련 기본지식.pdf...
Extracting from 대학생을위한실용금융(제4판)_금융감독원_ePDF(20240320).pdf...
Extracting from 대학생이 꼭 알아야할 금융이야기.pdf...
모든 PDF 텍스트 추출 완료 및 combined_text.txt 저장됨


In [31]:
def clean_text(text):
    # 줄바꿈, 탭을 공백으로 변경
    text = text.replace('\n', ' ').replace('\t', ' ')
    # 여러 공백을 한 칸 공백으로 변경
    text = re.sub(' +', ' ', text)
    # 특수문자 일부 제거 (필요에 따라 수정)
    text = re.sub(r'[■◆●▲▼◀▶★]', '', text)
    # 문장부호 앞뒤 공백 제거
    text = re.sub(r'\s*([,.?!])\s*', r'\1 ', text)
    return text.strip()


def split_korean_sentences(text):
    # 마침표, 느낌표, 물음표 뒤에 공백이 있거나 줄바꿈이 오는 곳에서 분리
    sentences = re.split(r'(?<=[.!?])\s+', text)
    return [s.strip() for s in sentences if s.strip()]

def preprocess_korean_sentences(sentences):
    cleaned = []
    for line in sentences:
        line = line.strip()
        if len(line) < 5:
            if re.match(r'^[\d\s\.\,\-\–\=\+\(\)\[\]\/\\·※▶▶\•a-zA-Z가-힣]{1,4}$', line):
                continue
        if re.match(r'^(그림|표|참고|출처|그림\s*\d+|표\s*\d+)', line):
            continue
        cleaned.append(line)
    return cleaned

processed_text = clean_text(full_text)
sentences = split_korean_sentences(processed_text)
filtered = preprocess_korean_sentences(sentences)

with open('processed_sentences.txt', 'w', encoding='utf-8') as f:
    for s in filtered:
        f.write(s + '\n')